In [1]:
from pyspark import SparkContext
from operator import add

In [ ]:
sc = SparkContext(appName="Large-Scale Customer Data Wrangling_RDD")

In [2]:
customer_lines = sc.textFile('customer.tbl.txt')
customer_lines = customer_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
customer = customer_lines.map(lambda x: x.split('|'))

lineitem_lines = sc.textFile('lineitem.tbl.txt')
lineitem_lines = lineitem_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
lineitem = lineitem_lines.map(lambda x: x.split('|'))

nation_lines = sc.textFile('nation.tbl.txt')
nation_lines = nation_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
nation = nation_lines.map(lambda x: x.split('|'))

part_lines = sc.textFile('part.tbl.txt')
part_lines = lineitem_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
part = part_lines.map(lambda x: x.split('|'))

partsupp_lines = sc.textFile('partsupp.tbl.txt')
partsupp_lines = partsupp_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
partsupp = partsupp_lines.map(lambda x: x.split('|'))

region_lines = sc.textFile('region.tbl.txt')
region_lines = region_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
region = region_lines.map(lambda x: x.split('|'))

supplier_lines = sc.textFile('supplier.tbl.txt')
supplier_lines = supplier_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
supplier = supplier_lines.map(lambda x: x.split('|'))

orders_lines = sc.textFile('orders.tbl.txt')
orders_lines = orders_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
orders = orders_lines.map(lambda x: x.split('|'))

In [3]:
# Question 1: What are the top-10 sold products?
top_10_sold_product = lineitem.map(lambda x:(x[1],1)).reduceByKey(add).top(10, lambda x:x[1])
top_10_sold_product

[('10620', 56),
 ('6140', 54),
 ('8051', 52),
 ('15584', 52),
 ('10715', 51),
 ('10597', 51),
 ('2292', 51),
 ('14422', 50),
 ('17670', 50),
 ('19444', 50)]

In [4]:
# Question 2: What are the top-10 customers based on the number of products ordered?
orders_sub = orders.map(lambda x:(x[0],x[1]))
lineitem_sub = lineitem.map(lambda x: (x[0],x[4]))
cust_quantity = orders_sub.join(lineitem_sub).map(lambda x:(x[1][0],float(x[1][1])))
top_10_customers = cust_quantity.reduceByKey(add).top(10, lambda x:x[1])
top_10_customers

[('8362', 4082.0),
 ('9454', 3870.0),
 ('346', 3817.0),
 ('6958', 3760.0),
 ('1105', 3737.0),
 ('14707', 3710.0),
 ('11998', 3709.0),
 ('14398', 3670.0),
 ('8542', 3660.0),
 ('8761', 3658.0)]

In [5]:
# Question 3: What are the top-10 customers that have ordered products from the same supplier
lineitem_sub2 = lineitem.map(lambda x: (x[0],x[2]))
cust_supp = orders_sub.join(lineitem_sub2).map(lambda x:((x[1][0],x[1][1]),1))
top_10_customers_same_supp = cust_supp.reduceByKey(add).top(10, lambda x:x[1])
top_10_customers_same_supp

[(('2767', '601'), 5),
 (('1567', '750'), 4),
 (('4177', '985'), 4),
 (('14581', '902'), 4),
 (('14875', '452'), 4),
 (('11515', '113'), 4),
 (('11354', '127'), 4),
 (('2119', '603'), 4),
 (('9562', '695'), 4),
 (('6868', '344'), 4)]

In [6]:
# Question 4: Who are the customers that have not ordered products from their own country and have ordered only foreign products.
customer_sub = customer.map(lambda x:(x[0],x[3]))
order_cust_nation = orders_sub.map(lambda x:(x[1],x[0])).join(customer_sub)

supplier_sub = supplier.map(lambda x: (x[0],x[3]))
supp_order_nation = lineitem_sub2.map(lambda x:(x[1],x[0])).join(supplier_sub)
order_supper_nation = supp_order_nation.map(lambda x: (x[1][0],(x[0],x[1][1])))

order_nation_cust_nation = order_supper_nation.join(order_cust_nation).map(lambda x: (x[0],(x[1][0][1],x[1][1][0],x[1][1][1])))
cust_OwnCountryCnt = order_nation_cust_nation.map(lambda x: (x[1][1],1) if x[1][0]==x[1][2] else (x[1][1],0)).reduceByKey(add)

cust_onlyforeign = cust_OwnCountryCnt.filter(lambda x: x[1]==0).map(lambda x:x[0])
cust_onlyforeign.collect()

['158405',
 '245057',
 '282563',
 '316230',
 '90849',
 '289697',
 '486917',
 '382343',
 '48002',
 '309730',
 '360038',
 '372961',
 '523136',
 '115489',
 '185127',
 '517189',
 '85633',
 '171554',
 '431622',
 '530756',
 '587590',
 '56833',
 '269894',
 '493636',
 '581379',
 '358055',
 '351202',
 '64647',
 '410369',
 '467779',
 '560864',
 '300097',
 '439461',
 '483622',
 '169701',
 '211621',
 '269729',
 '429349',
 '577666',
 '223875',
 '402177',
 '540550',
 '378758',
 '495460',
 '145539',
 '199174',
 '486753',
 '32513',
 '594529',
 '48134',
 '255876',
 '251552',
 '491297',
 '243844',
 '392448',
 '404551',
 '537543',
 '13920',
 '362595',
 '564582',
 '592162',
 '322021',
 '319555',
 '523590',
 '3270',
 '220676',
 '79808',
 '134210',
 '146337',
 '310119',
 '333251',
 '35238',
 '99812',
 '247559',
 '161639',
 '182757',
 '266977',
 '309280',
 '371591',
 '448871',
 '559296',
 '590784',
 '376515',
 '290727',
 '427429',
 '145765',
 '315813',
 '598726',
 '79205',
 '574661',
 '36039',
 '509088',
 '2

In [7]:
# Question 5: Which top 3 countries produced most of the products that are ordered
lineitem_sub3 = lineitem.map(lambda x: (x[2],x[1]))
nation_part = lineitem_sub3.join(supplier_sub).map(lambda x: (x[1][1],x[1][0]))
nation_partcnt = nation_part.map(lambda x:(x[0],1)).reduceByKey(add).top(3, lambda x:x[1])
nation_partcnt

[('18', 31483), ('7', 29975), ('22', 28316)]

In [10]:
# Question 6: Who are the top-10 similar customers based of their orders? 
# (Use Jaccard similarity to calculate the similarity) 
# Consider only customers that have ordered at least 10 products. First collect all the products that each customer ordered.
lineitem_sub4 = lineitem.map(lambda x:(x[0],x[1]))
cust_part = orders_sub.join(lineitem_sub4).map(lambda x: (x[1][0],[x[1][1]]))

cust_partlist = cust_part.reduceByKey(add).filter(lambda x: len(x[1])>=10)

9973

In [41]:
crossjoin_cust_partlist = cust_partlist.cartesian(cust_partlist).filter(lambda x: float(x[0][0]) < float(x[1][0]))

In [37]:
pair_jaccard = crossjoin_cust_partlist.map(lambda x: ((x[0][0],x[1][0]),(len(set(x[0][1])&set(x[1][1])))/len(set(x[0][1])|set(x[1][1]))))

In [38]:
top_10_similar_customers = pair_jaccard.top(10, lambda x: x[1])
top_10_similar_customers

[(('4808', '10901'), 0.06666666666666667),
 (('5390', '7532'), 0.06451612903225806),
 (('2489', '4283'), 0.06349206349206349),
 (('2768', '4385'), 0.0625),
 (('5462', '7739'), 0.0625),
 (('944', '11402'), 0.061224489795918366),
 (('668', '12746'), 0.06060606060606061),
 (('29', '6788'), 0.06060606060606061),
 (('1976', '9740'), 0.058823529411764705),
 (('10418', '11432'), 0.058823529411764705)]

In [46]:
# Question 7: What are the top-10 products pairs that the customer ordered mostly together?
cust_part2 = orders_sub.join(lineitem_sub4).map(lambda x: (x[1][0],x[1][1]))


[('4450', '10857')]

In [51]:
cust_part_part = cust_part2.join(cust_part2).filter(lambda x: float(x[1][0]) < float(x[1][1]))
pair_cnt = cust_part_part.map(lambda x: (x[1],1))

In [52]:
top_10_products_pairs = pair_cnt.reduceByKey(add).top(10, lambda x:x[1])
top_10_products_pairs

[(('9021', '18846'), 6),
 (('9849', '12884'), 5),
 (('15294', '19324'), 5),
 (('11514', '11566'), 5),
 (('5318', '12705'), 5),
 (('4994', '13106'), 5),
 (('13405', '13875'), 5),
 (('3376', '9230'), 5),
 (('844', '13738'), 5),
 (('2100', '18702'), 5)]

In [ ]:
sc.stop()